# Set up

In [ ]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r
#

Cloning into 'assignment_1'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 150 (delta 35), reused 22 (delta 22), pack-reused 106
Receiving objects: 100% (150/150), 6.79 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (64/64), done.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [ ]:
import json
from google.colab import files
import pandas as pd
import os
from collections import defaultdict
import numpy as np

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [ ]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  vocab = set()

  folder_path = 'data'
  csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
  for file in csv_files:
    file_path = folder_path + "/" + file
    df = pd.read_csv(file_path)
    tweets = ''.join(df.iloc[:, 1])
    vocab |= {*tweets}

  vocab_list = list(vocab)
  start_token, end_token = "<start>", "<end>"
  vocab_list.append(start_token)
  vocab_list.append(end_token)
  return vocab_list

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [ ]:
def lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
    '''
    Return a language model for the given lang and n-gram (n), with an option for smoothing.
    :param lang: the language of the model
    :param n: the n_gram value
    :param smoothed: boolean indicating whether to apply smoothing
    :return: a dictionary where the keys are n-1 grams and the values are dictionaries
    '''


    df = pd.read_csv(f"data/{lang}.csv")
    vocab = preprocess()
    vocab_size = len(vocab)

    model: dict[str, dict[str, float]] = {}

    tweets = df.iloc[:, 1].tolist()

    formatted_tweets = [f'{"<start>" * (n-1)}{tweet}<end>' for tweet in tweets]
    corpus = ''.join(formatted_tweets)
    del tweets, formatted_tweets



    tokens = []
    i = 0
    while i < len(corpus):
        if corpus[i:i+7] == '<start>':
            tokens.append('<start>')
            i += 7
        elif corpus[i:i+5] == '<end>':
            tokens.append('<end>')
            i += 5
        else:
            tokens.append(corpus[i])
            i += 1

    del corpus
    ngram_counter: dict[str, int] = {}
    context_counter: dict[str, int] = {}

    for i in range(len(tokens) - n + 1):
        ngram = ''.join(tokens[i:i+n])
        if ngram not in ngram_counter:
          ngram_counter[ngram] = 1
        else:
          ngram_counter[ngram] += 1

    for i in range(len(tokens) - n + 2):
        context = ''.join(tokens[i:i+n-1])  # Join tokens to form n-gram
        if '<end>' not in context:
          if context not in model:
           context_counter[context] = 1
           model[context] = {}
          else:
           context_counter[context] += 1

    del tokens

    for seq in ngram_counter:

        if seq[-5:] == '<end>':
            context = seq[:-5]
            next_token = '<end>'
        else:
            context = seq[:-1]
            next_token = seq[-1]
        if context in model:
          context_count = context_counter[context]
          if smoothed:
            model[context][next_token] = (ngram_counter[seq] + 1)/(context_count + vocab_size)
          else:
            model[context][next_token] = ngram_counter[seq]/context_count

    for context, tokens in model.items():
      total_count = sum(tokens.values())
      for token in tokens:
        tokens[token] /= total_count

    if smoothed:
       model['<unk>'] = 1/vocab_size

    return model

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [ ]:
def eval(model: dict, target_lang: str, n: int) -> float:

  df = pd.read_csv(f"data/{target_lang}.csv")
  tweets = df.iloc[:, 1].tolist()

  sum_entropy = 0
  for tweet in tweets:
    prob = 0
    if tweet[0] not in model[f'{"<start>" * (n-1)}']:
      prob += np.log2(model['<unk>'])
    else:
      prob += np.log2(model[f'{"<start>" * (n-1)}'][tweet[0]])
    for i in range(len(tweet) - n + 1):
        seq = ''.join(tweet[i:i+n])
        if seq[:n-1] in model:
          if seq[-1] not in model[seq[:n-1]]:
            prob += np.log2(model['<unk>'])
          else:
            prob += np.log2(model[seq[:n-1]][seq[-1]])
        else:
          prob += np.log2(model['<unk>'])

    entropy = (-1)/(len(tweet)) * prob
    sum_entropy += entropy

  avg_entropy = sum_entropy/len(tweets)
  return 2**avg_entropy

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [ ]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  folder_path = 'data'
  csv_files = [file[:-4] for file in os.listdir(folder_path) if file.endswith('.csv')]

  data = []

  for source in csv_files:
    for n in range(1, 5):
      model = lm(lang=source, n=n, smoothed = True)

      for target in csv_files:
        perplexity = eval(model, target, n)
        data.append({
          "source": source,
          "target": target,
          "n": n,
          "perplexity": perplexity
          })

      model = None
  df = pd.DataFrame(data)
  return df

match()

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

Note #2 - if you use an <END> token in your vocabulary, then once the <END> token is generated by the model, you should stop generating text.

In [ ]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  np.random.seed(r)

  model = lm(lang = lang, n = n, smoothed = False)
  tokens = list(prompt)
  context = ''.join(tokens[-(n-1):]) if n>1 else ''
  for _ in range(number_of_tokens):
    if '<end>' in context:
      break
    next_tokens, probs = zip(*model[context].items())
    probabilities = np.array(probs)
    next_token = np.random.choice(next_tokens, p=probabilities)
    if next_token == '<end>':
      break
    tokens.append(next_token)
    context = ''.join((context + next_token)[-n+1:]) if n>1 else ''

  return "".join(tokens)

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [ ]:
generate('en', 1, "I am", 20, 5)

'I amD.lIntudyE pe .oeytt'

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [ ]:
####################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_lm():
    return {
        'english_2_gram_length': len(lm('en', 2, True)),
        'english_3_gram_length': len(lm('en', 3, True)),
        'french_3_gram_length': len(lm('fr', 3, True)),
        'spanish_3_gram_length': len(lm('es', 3, True)),
    }

def test_eval():
    return {
        'en_en': eval(lm('en', 3, True), 'en', 3),
        'en_fr': eval(lm('en', 3, True), 'fr', 3),
        'en_tl': eval(lm('en', 3, True), 'tl', 3),
        'en_nl': eval(lm('en', 3, True), 'nl', 3),
    }

def test_match():
    df = match()
    return {
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_1_gram': generate('en', 1, "I", 20, 5),
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')
####################

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l